In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [12]:
visits = pd.read_csv('./visits', delimiter='\t')
purchases = pd.read_csv('./purchases', delimiter='\t')

In [13]:
# конвертируем даты
visits['end_dt'] = visits['end_ts'].apply(lambda x: datetime.strptime(x.split('T')[0], '%Y-%m-%d'))
visits['end_ts'] = visits['end_ts'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
visits['start_ts'] = visits['start_ts'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
visits['start_dt'] = visits['start_dt'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
purchases['buy_ts'] = purchases['buy_ts'].apply(lambda x: datetime.strptime(x, '%d.%m.%Y %H:%M:%S'))
purchases['revenue'] = purchases['revenue'].apply(lambda x: float(x.replace(',', '.')))
purchases['month'] = purchases['buy_ts'].apply(lambda x: x.month)

In [14]:
v = visits.rename({'device': 'amount'}, axis='columns').groupby(by='uid').agg({
    'start_ts': 'min',
    'end_ts': 'max',
    'amount': 'count'
})
print("Количество таких визитов - ", len(v))
a = list(v.index)
b = list(purchases[purchases['uid'].isin(a)].uid.unique())
print("Конверсия (клиент -> покупатель)", round(len(b) / len(a), 2))
print("Среднее количество посещений:", round(v.amount.mean(), 2))

uids = purchases[(purchases['month'].isin([10,11,12,1])) & (purchases['uid'].isin(b))].uid.unique()
tmp =  purchases[purchases['uid'].isin(uids)].copy()
tmp['end'] = tmp['buy_ts']
groupedData = tmp.sort_values(by='buy_ts').rename({
    'month': 'amount', 
    'buy_ts': 'start'
    }, axis='columns').groupby(by='uid').agg({
    'start': 'min', 
    'end': 'max',
    'amount': 'count', 
    'revenue': 'sum'})
#     groupedData = groupedData[groupedData['amount'] <= 30] # Фильтр на китов
    
# Количество заходов до первой покупки:
a = visits[visits['uid'].isin(groupedData.index)].rename({'device': 'amount'}, axis='columns').groupby(by='uid').agg({
    'start_ts': 'min',
    'end_ts': 'max',
    'amount': 'count'
})
a = a.join(groupedData, how='inner', lsuffix='-visits', rsuffix='-purchases')
a['time_after_visit'] = a['start'] - a['start_ts']
a['time_after_visit'] = a.time_after_visit.apply(lambda x: x.days)
print('Дней с первого посещения до первой покупки: ', round(a.time_after_visit.mean(), 2))
print("Среднее количество визитов у тех, кто стал клиентом", round(a['amount-visits'].mean(), 2))
print('Отношение визит / покупка:', round(a['amount-visits'].mean() / a['amount-purchases'].mean(), 2))

groupedData['delta'] = groupedData['end'] - groupedData['start']
groupedData['delta'] = groupedData.delta.apply(lambda x: x.days // 30) # количество месяцев

print("ARPU: ", round(groupedData.revenue.mean(), 2), 'Lifetime: ', round(groupedData.delta.mean(), 2), ' месяцев')
print("LTV = ARPU * Lifetime = ", round(groupedData.revenue.mean() * groupedData.delta.mean(), 2))
print('\n--------------------------------------\n')

Количество таких визитов -  380121
Конверсия (клиент -> покупатель) 0.15
Среднее количество посещений: 1.9
Дней с первого посещения до первой покупки:  14.43
Среднее количество визитов у тех, кто стал клиентом 4.96
Отношение визит / покупка: 3.15
ARPU:  8.66 Lifetime:  0.38  месяцев
LTV = ARPU * Lifetime =  3.3

--------------------------------------



In [9]:
# Рассчет без джойнов
for t in ['desktop', 'touch']:
    print(t.upper())
    v = visits[visits['device'] == t].rename({'device': 'amount'}, axis='columns').groupby(by='uid').agg({
        'start_ts': 'min',
        'end_ts': 'max',
        'amount': 'count'
    })
    print("Количество таких визитов - ", len(v))
    a = list(v.index)
    b = list(purchases[purchases['uid'].isin(a)].uid.unique())
    print("Конверсия (клиент -> покупатель)", round(len(b) / len(a), 2))
    print("Среднее количество посещений:", round(v.amount.mean(), 2))
    
    uids = purchases[(purchases['month'].isin([10,11,12,1])) & (purchases['uid'].isin(b))].uid.unique()
    tmp =  purchases[purchases['uid'].isin(uids)].copy()
    tmp['end'] = tmp['buy_ts']
    groupedData = tmp.sort_values(by='buy_ts').rename({
        'month': 'amount', 
        'buy_ts': 'start'
        }, axis='columns').groupby(by='uid').agg({
        'start': 'min', 
        'end': 'max',
        'amount': 'count', 
        'revenue': 'sum'})
#     groupedData = groupedData[groupedData['amount'] <= 30] # Фильтр на китов
    
    # Количество заходов до первой покупки:
    a = visits[visits['uid'].isin(groupedData.index)].rename({'device': 'amount'}, axis='columns').groupby(by='uid').agg({
        'start_ts': 'min',
        'end_ts': 'max',
        'amount': 'count'
    })
    a = a.join(groupedData, how='inner', lsuffix='-visits', rsuffix='-purchases')
    a['time_after_visit'] = a['start'] - a['start_ts']
    a['time_after_visit'] = a.time_after_visit.apply(lambda x: x.days)
    print('Дней с первого посещения до первой покупки: ', round(a.time_after_visit.mean(), 2))
    print("Среднее количество визитов у тех, кто стал клиентом", round(a['amount-visits'].mean(), 2))
    print('Отношение визит / покупка:', round(a['amount-visits'].mean() / a['amount-purchases'].mean(), 2))

    groupedData['delta'] = groupedData['end'] - groupedData['start']
    groupedData['delta'] = groupedData.delta.apply(lambda x: x.days // 30) # количество месяцев

    print("ARPU: ", round(groupedData.revenue.mean(), 2), 'Lifetime: ', round(groupedData.delta.mean(), 2), ' месяцев')
    print("LTV = ARPU * Lifetime = ", round(groupedData.revenue.mean() * groupedData.delta.mean(), 2))
    print('\n--------------------------------------\n')


DESKTOP
Количество таких визитов -  275274
Конверсия (клиент -> покупатель) 0.17
Среднее количество посещений: 1.92
Дней с первого посещения до первой покупки:  14.91
Среднее количество визитов у тех, кто стал клиентом 5.2
Отношение визит / покупка: 3.2
ARPU:  9.26 Lifetime:  0.42  месяцев
LTV = ARPU * Lifetime =  3.85

--------------------------------------

TOUCH
Количество таких визитов -  122676
Конверсия (клиент -> покупатель) 0.12
Среднее количество посещений: 1.58
Дней с первого посещения до первой покупки:  21.89
Среднее количество визитов у тех, кто стал клиентом 7.02
Отношение визит / покупка: 4.26
ARPU:  9.82 Lifetime:  0.51  месяцев
LTV = ARPU * Lifetime =  5.0

--------------------------------------



In [10]:
for t in [ 2,  4,  1,  9,  5,  3, 10,  7,  6]:
    print("Источник: ", t)
    v = visits[visits['source_id'] == t].rename({'device': 'amount'}, axis='columns').groupby(by='uid').agg({
        'start_ts': 'min',
        'end_ts': 'max',
        'amount': 'count'
    })
    print("Количество таких визитов - ", len(v))
    a = list(v.index)
    b = list(purchases[purchases['uid'].isin(a)].uid.unique())
    print("Конверсия (клиент -> покупатель)", round(len(b) / len(a), 2))
    print("Среднее количество посещений:", round(v.amount.mean(), 2))
    
    uids = purchases[(purchases['month'].isin([10,11,12,1])) & (purchases['uid'].isin(b))].uid.unique()
    tmp =  purchases[purchases['uid'].isin(uids)].copy()
    tmp['end'] = tmp['buy_ts']
    groupedData = tmp.sort_values(by='buy_ts').rename({
        'month': 'amount', 
        'buy_ts': 'start'
        }, axis='columns').groupby(by='uid').agg({
        'start': 'min', 
        'end': 'max',
        'amount': 'count', 
        'revenue': 'sum'})
    groupedData = groupedData[groupedData['amount'] <= 30] # Фильтр на китов
    
    # Количество заходов до первой покупки:
    a = visits[visits['uid'].isin(groupedData.index)].rename({'device': 'amount'}, axis='columns').groupby(by='uid').agg({
        'start_ts': 'min',
        'end_ts': 'max',
        'amount': 'count'
    })
    a = a.join(groupedData, how='inner', lsuffix='-visits', rsuffix='-purchases')
    a['time_after_visit'] = a['start'] - a['start_ts']
    a['time_after_visit'] = a.time_after_visit.apply(lambda x: x.days)
    print('Дней с первого посещения до первой покупки: ', round(a.time_after_visit.mean(), 2))
    print("Среднее количество визитов у тех, кто стал клиентом", round(a['amount-visits'].mean(), 2))
    print('Отношение визит / покупка:', round(a['amount-visits'].mean() / a['amount-purchases'].mean(), 2))

    groupedData['delta'] = groupedData['end'] - groupedData['start']
    groupedData['delta'] = groupedData.delta.apply(lambda x: x.days // 30) # количество месяцев

    print("ARPU: ", round(groupedData.revenue.mean(), 2), 'Lifetime: ', round(groupedData.delta.mean(), 2), ' месяцев')
    print("LTV = ARPU * Lifetime = ", round(groupedData.revenue.mean() * groupedData.delta.mean(), 2))
    print('\n--------------------------------------\n')

Источник:  2
Количество таких визитов -  44521
Конверсия (клиент -> покупатель) 0.29
Среднее количество посещений: 2.16
Дней с первого посещения до первой покупки:  25.24
Среднее количество визитов у тех, кто стал клиентом 9.36
Отношение визит / покупка: 4.99
ARPU:  11.92 Lifetime:  0.8  месяцев
LTV = ARPU * Lifetime =  9.59

--------------------------------------

Источник:  4
Количество таких визитов -  150820
Конверсия (клиент -> покупатель) 0.17
Среднее количество посещений: 1.35
Дней с первого посещения до первой покупки:  19.68
Среднее количество визитов у тех, кто стал клиентом 5.74
Отношение визит / покупка: 3.8
ARPU:  6.79 Lifetime:  0.52  месяцев
LTV = ARPU * Lifetime =  3.51

--------------------------------------

Источник:  1
Количество таких визитов -  32811
Конверсия (клиент -> покупатель) 0.39
Среднее количество посещений: 2.1
Дней с первого посещения до первой покупки:  23.02
Среднее количество визитов у тех, кто стал клиентом 9.51
Отношение визит / покупка: 5.15
ARPU:

In [ ]:
193442905 